Viterbi algorithm

In [9]:
'''My Own Implementation!'''

def viterbi(obs, states, start_pro, trans_pro, emit_pro):
    # state sequence probability
    seq_pro = [{}]

    # start states probability (t = 0)
    for state in states:
        seq_pro[0][state] = start_pro[state] * emit_pro[state][obs[0]]

    # transition states probatility (t > 0)
    for t in range(1,len(obs)):
        seq_pro.append({})
        for next_state in states:
            max_pro = max([seq_pro[t-1][cur_state] * trans_pro[cur_state][next_state] * emit_pro[next_state][obs[t]] for cur_state in states])
            seq_pro[t][next_state] = max_pro
    return seq_pro

def table_creater(seq_pro):
    '''Turn "state sequence probability" into "Trellis diagram".'''
    print
    print " " * 8,
    for i in range(len(seq_pro)):
        print "%8d" % i,
    print

    for state in seq_pro[0].keys():
        print "%8s" % state,
        for t in range(len(seq_pro)):
            print "%8.7s" % ("%f" % seq_pro[t][state]),
        print
    print

def path_finder(obs, seq_pro):
    '''Find The most probable path.'''
    path = []
    for t in range(len(seq_pro)):
        values = list(seq_pro[t].values())
        keys = list(seq_pro[t].keys())
        path.append(keys[values.index(max(values))])
    print "The observations are:",
    for i in obs:
        print i,
    print
    print "The most probable hidden path is:",
    for i in path:
        print i,

observations = ['dry', 'drizzling', 'dry', 'drizzling', 'rainy']
total_states = ['clear', 'cloudy', 'overcast']
start_probability = {'clear': 0.5, 'cloudy': 0.3, 'overcast': 0.2}
trans_probability = {'clear': {'clear': 0.5, 'cloudy': 0.3, 'overcast': 0.2},
                     'cloudy': {'clear': 0.2, 'cloudy': 0.5, 'overcast': 0.3},
                     'overcast': {'clear': 0.2, 'cloudy': 0.3, 'overcast': 0.5}}
emission_probability = {'clear': {'dry': 0.6, 'drizzling': 0.2, 'rainy': 0.2},
                        'cloudy': {'dry': 0.2, 'drizzling': 0.5, 'rainy': 0.3},
                        'overcast': {'dry': 0.2, 'drizzling': 0.3, 'rainy': 0.5}}

V = viterbi(observations, total_states, start_probability, trans_probability, emission_probability)
table_creater(V)
path_finder(observations, V)


                0        1        2        3        4
   clear  0.30000  0.03000  0.00900  0.00090  0.00009
overcast  0.04000  0.01800  0.00270  0.00054  0.00020
  cloudy  0.06000  0.04500  0.00450  0.00135  0.00020

The observations are: dry drizzling dry drizzling rainy
The most probable hidden path is: clear cloudy clear cloudy overcast


In [15]:
#From open source http://stackoverflow.com/questions/7396849/convert-binary-to-ascii-and-vice-versa
import binascii

def text_to_bits(text, encoding='utf-8', errors='surrogatepass'):
    bits = bin(int(binascii.hexlify(text.encode(encoding, errors)), 16))[2:]
    return bits.zfill(8 * ((len(bits) + 7) // 8))

def text_from_bits(bits, encoding='utf-8', errors='surrogatepass'):
    n = int(bits, 2)
    return int2bytes(n).decode(encoding, errors)

def int2bytes(i):
    hex_string = '%x' % i
    n = len(hex_string)
    return binascii.unhexlify(hex_string.zfill(n + (n & 1)))

def viterbi_enc(fsm, s0, x):
    y = []
    ps = s0
    for symbol in x:
        y.append(fsm[ps][symbol]['out'])
        ps = fsm[ps][symbol]['ns']
    return y

import random

def add_noise(error_rate, x):
    random.seed(10000)
    y = []
    for code in x:
        new_code = ''
        for b in code:
            if random.random() < error_rate:
                new_b = '0' if b =='1' else '1'
            else:
                new_b = b
            new_code = new_code + new_b
        y.append(new_code)
    return y

def ham_dist(a, b):
    d = 0
    for n in range(len(a)):
        if a[n] != b[n]:
            d += 1
    return d

def viterbi_dec(fsm_rev, s0, y):
    dp_array = [{}]
    path = {}
    for s in fsm_rev.keys():
        dp_array[0][s] = 0 if s ==s0 else 9999
        path[s] = [s]
    for i in range(len(y)):
        dp_array.append({})
        new_path = {}
        for s in fsm_rev.keys():
            min_dist = 999999
            cur_dist = 999999
            for s_prev in fsm_rev[s].keys():
                dist = ham_dist(y[i], fsm_rev[s][s_prev]['out'])
                cur_dist = dp_array[i][s_prev] + dist
                if cur_dist < min_dist:
                    min_dist = cur_dist
                    min_state = s_prev
            dp_array[i+1][s] = min_dist
            new_path[s] = path[min_state] + [s]
        path = new_path
    best_dist, best_s = min((dp_array[-1][s], s) for s in dp_array[-1].keys())
    best_p = path[best_s]
    ml_code = []
    ml_in = []
    for i in range(len(best_p)-1, 0, -1):
        ml_code = [fsm_rev[best_p[i]][best_p[i-1]]['out']] + ml_code
        ml_in =   [fsm_rev[best_p[i]][best_p[i-1]]['in']] + ml_in
    return ml_code, ml_in

def viterbi_dec_limit_buff(fsm_rev, s0, buff_limit, y):
    dp_array = [{}]
    path = {}
    ml = []
    ml_in = []
    for s in fsm_rev.keys():
        dp_array[0][s] = 0 if s ==s0 else 9999
        path[s] = [s]
    for i in range(len(y)):
        dp_array.append({})
        new_path = {}
        for s in fsm_rev.keys():
            min_dist = 999999
            cur_dist = 999999
            for s_prev in fsm_rev[s].keys():
                dist = ham_dist(y[i], fsm_rev[s][s_prev]['out'])
                cur_dist = dp_array[-2][s_prev] + dist
                if cur_dist < min_dist:
                    min_dist = cur_dist
                    min_state = s_prev
            dp_array[-1][s] = min_dist
            new_path[s] = path[min_state] + [s]
        path = new_path
        if (len(dp_array) > buff_limit):
            best_dist, best_s = min((dp_array[-1][s], s) for s in dp_array[-1])
            best_p = path[best_s]
            ml += [fsm_rev[best_p[1]][best_p[0]]['out']]
            ml_in += [fsm_rev[best_p[1]][best_p[0]]['in']]
            dp_array.pop(0)
            for p in path:
                path[p].pop(0)
        best_dist, best_s = min((dp_array[-1][s], s) for s in dp_array[-1])
        best_p = path[best_s]   
        for i in range(1, len(best_p)):
            ml += [fsm_rev[best_p[i]][best_p[i-1]]['out']]
            ml_in += [fsm_rev[best_p[i]][best_p[i-1]]['in']]
    return dp_array, path, ml, ml_in

fsm = {'a':{'0':{'ns':'a', 'out':'00'},
            '1':{'ns':'b', 'out':'11'}},
       'b':{'0':{'ns':'c', 'out':'10'},
            '1':{'ns':'d', 'out':'01'}},
       'c':{'0':{'ns':'a', 'out':'11'},
            '1':{'ns':'b', 'out':'00'}},
       'd':{'0':{'ns':'c', 'out':'01'},
            '1':{'ns':'d', 'out':'10'}}}
fsm_rev = {'a':{'a':{'in':'0', 'out':'00'},
                'c':{'in':'0', 'out':'11'}},
           'b':{'a':{'in':'1', 'out':'11'},
                'c':{'in':'1', 'out':'00'}},
           'c':{'b':{'in':'0', 'out':'10'},
                'd':{'in':'0', 'out':'01'}},
           'd':{'b':{'in':'1', 'out':'01'},
                'd':{'in':'1', 'out':'10'}}}

x = text_to_bits('TensorFlow')
y = viterbi_enc(fsm,'a',x)
print ("y", y)
z = add_noise(.01, y)
print ("z", z)
mL_code, mL_in = viterbi_dec(fsm_rev, 'a', z)
print(mL_in)
dec_msg = text_from_bits(''.join(mL_in))
print ("Decoded MSG:", dec_msg)
print(['D' if a != b else '' for a,b in zip(z, mL_code)])

('y', ['00', '11', '10', '00', '10', '00', '10', '11', '00', '11', '01', '01', '11', '11', '10', '00', '10', '00', '01', '01', '00', '01', '10', '01', '11', '11', '01', '10', '01', '11', '11', '01', '01', '00', '01', '01', '00', '01', '10', '10', '01', '00', '01', '10', '01', '11', '11', '10', '11', '11', '10', '11', '00', '11', '01', '01', '11', '11', '01', '01', '00', '01', '01', '11', '00', '11', '01', '01', '00', '01', '10', '10', '01', '00', '01', '10', '01', '00', '01', '10'])
('z', ['00', '11', '10', '01', '10', '00', '10', '11', '10', '11', '01', '01', '11', '11', '10', '00', '10', '00', '01', '01', '00', '01', '10', '01', '11', '11', '01', '10', '01', '11', '11', '01', '01', '00', '01', '01', '00', '01', '10', '10', '01', '00', '01', '10', '01', '11', '10', '10', '11', '11', '10', '11', '00', '11', '01', '01', '11', '11', '01', '01', '00', '01', '01', '11', '00', '11', '01', '01', '00', '01', '10', '10', '01', '00', '01', '10', '00', '00', '01', '10'])
['0', '1', '0', '1', '0'

Pagerank

In [ ]:
#from https://gist.github.com/diogojc/1338222
import numpy as np
from numpy.sparse import csc_matrix

def pageRank(G, s = .85, maxerr = .001):
    """
    Computes the pagerank for each of the n states.
    Used in webpage ranking and text summarization using unweighted
    or weighted transitions respectively.
    Args
    ----------
    G: matrix representing state transitions
       Gij can be a boolean or non negative real number representing the
       transition weight from state i to j.
    Kwargs
    ----------
    s: probability of following a transition. 1-s probability of teleporting
       to another state. Defaults to 0.85
    maxerr: if the sum of pageranks between iterations is bellow this we will
            have converged. Defaults to 0.001
    """
    n = G.shape[0]

    # transform G into markov matrix M
    M = csc_matrix(G,dtype=np.float)
    rsums = np.array(M.sum(1))[:,0]
    ri, ci = M.nonzero()
    M.data /= rsums[ri]

    # bool array of sink states
    sink = rsums==0

    # Compute pagerank r until we converge
    ro, r = np.zeros(n), np.ones(n)
    while np.sum(np.abs(r-ro)) > maxerr:
        ro = r.copy()
        # calculate each pagerank at a time
        for i in xrange(0,n):
            # inlinks of state i
            Ii = np.array(M[:,i].todense())[:,0]
            # account for sink states
            Si = sink / float(n)
            # account for teleportation to state i
            Ti = np.ones(n) / float(n)

            r[i] = ro.dot( Ii*s + Si*s + Ti*(1-s) )

    # return normalized pagerank
    return r/sum(r)




if __name__=='__main__':
    # Example extracted from 'Introduction to Information Retrieval'
    G = np.array([[0,0,1,0,0,0,0],
                  [0,1,1,0,0,0,0],
                  [1,0,1,1,0,0,0],
                  [0,0,0,1,1,0,0],
                  [0,0,0,0,0,0,1],
                  [0,0,0,0,0,1,1],
                  [0,0,0,1,1,0,1]])

    print pageRank(G,s=.86)

Maximum Flow Problem

In [23]:
# Ford–Fulkerson algorithm
class Edge(object):
    def __init__(self, u, v, w):
        self.source = u
        self.sink = v  
        self.capacity = w
    def __repr__(self):
        return "%s->%s:%s" % (self.source, self.sink, self.capacity)

class FlowNetwork(object):
    def __init__(self):
        self.adj = {}
        self.flow = {}
 
    def add_vertex(self, vertex):
        self.adj[vertex] = []
 
    def get_edges(self, v):
        return self.adj[v]
 
    def add_edge(self, u, v, w=0):
        if u == v:
            raise ValueError("u == v")
        edge = Edge(u,v,w)
        redge = Edge(v,u,0)
        edge.redge = redge
        redge.redge = edge
        self.adj[u].append(edge)
        self.adj[v].append(redge)
        self.flow[edge] = 0
        self.flow[redge] = 0
 
    def find_path(self, source, sink, path):
        if source == sink:
            return path
        for edge in self.get_edges(source):
            residual = edge.capacity - self.flow[edge]
            if residual > 0 and edge not in path:
                result = self.find_path( edge.sink, sink, path + [edge]) 
                if result != None:
                    return result
 
    def max_flow(self, source, sink):
        path = self.find_path(source, sink, [])
        while path != None:
            residuals = [edge.capacity - self.flow[edge] for edge in path]
            flow = min(residuals)
            for edge in path:
                self.flow[edge] += flow
                self.flow[edge.redge] -= flow
            path = self.find_path(source, sink, [])
        return sum(self.flow[edge] for edge in self.get_edges(source))

g = FlowNetwork()
[g.add_vertex(v) for v in "sopqrt"]
g.add_edge('s','o',3)
g.add_edge('s','p',3)
g.add_edge('o','p',2)
g.add_edge('o','q',3)
g.add_edge('p','r',2)
g.add_edge('r','t',3)
g.add_edge('q','r',4)
g.add_edge('q','t',2)
print (g.max_flow('s','t'))

5
